In [5]:
!pip install pandas

!pip install seaborn
!pip install numpy

!pip install matplotlib

!pip install scikit-learn

In [9]:
import matplotlib.pyplot as plt

import matplotlib

import seaborn as sns

import numpy as np
import numpy.linalg as np_la
from sympy import Matrix

import pandas as pd

from sklearn import neighbors, metrics

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from collections import Counter

import re

from sklearn.covariance import EllipticEnvelope

from sklearn.linear_model import LinearRegression

from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# functions

In [19]:
preproc_data = pd.read_csv("pre-processing_BankChurners.csv")
preproc_data_x = preproc_data.drop(["Attrition_Flag",
                       "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
                       "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
                       ], axis=1)
x = preproc_data_x.values
y = preproc_data["Attrition_Flag"].values
preproc_data_x

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,-0.165261,1.060261,0.503636,-0.893897,0.522808,0.425471,-0.263546,0.384495,0.763915,-1.327615,0.492010,0.447062,-0.473285,0.489410,2.666758,-0.960392,-0.975049,3.892696,-0.776043
1,0.583481,1.060261,0.503636,0.593115,0.522808,1.308928,-0.263546,0.008956,0.120645,-1.327615,-2.220431,-0.573420,-1.426735,-0.445377,8.499347,-0.741457,-1.913197,6.910076,-0.997334
2,-0.789213,-0.943070,1.273662,-0.893897,0.522808,-0.850632,-0.263546,-0.241404,-0.522625,1.643518,-1.316284,-0.584975,1.661853,-0.733868,2.991048,-0.952436,-1.913197,6.910076,1.759732
3,-0.789213,1.060261,0.503636,-1.637403,0.522808,0.425471,-0.263546,-1.868743,0.763915,-1.327615,-2.220431,-0.430577,-1.426735,-0.302563,6.558239,-1.057042,-1.572052,7.621803,-0.997334
4,-0.290051,1.060261,-0.266389,0.593115,0.522808,-0.163500,-0.263546,0.008956,-0.522625,-1.327615,-0.412137,-0.508271,0.103448,-0.517445,2.856699,-0.976893,-1.742625,0.572725,0.130886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10115,0.458691,1.060261,-0.266389,0.593115,-1.135236,-0.163500,-0.263546,0.509675,-0.522625,-0.337237,0.492010,-0.509042,0.844611,-0.584671,-0.261119,3.262728,2.223186,0.619605,0.678672
10116,-0.664422,1.060261,-0.266389,0.593115,2.180853,-0.163500,-0.263546,-1.368023,0.120645,-0.337237,0.492010,-0.478888,1.255686,-0.591383,0.206786,1.284945,0.176316,-0.121955,0.856430
10117,-0.290051,-0.943070,-1.036415,-0.893897,0.522808,-0.850632,-0.263546,0.008956,0.763915,0.653140,1.396157,-0.354313,-1.426735,-0.226314,0.276276,1.734897,-0.207472,0.453393,-0.997334
10118,-2.037116,1.060261,-0.266389,0.593115,0.522808,-0.163500,-0.263546,0.008956,0.120645,0.653140,0.492010,-0.368399,-1.426735,-0.240398,-1.039415,1.176215,-0.122186,0.044257,-0.997334


array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

# data type identifying

In [3]:

categorical_features = ['Attrition_Flag', 'Gender', 'Marital_Status']
ordinal_features = ["Dependent_count", "Education_Level", "Card_Category",
                "Income_Category", "Total_Relationship_Count",
                "Months_Inactive_12_mon", "Contacts_Count_12_mon"]
numerical_features = ['Credit_Limit', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1',
                      'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
                      'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                      'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
                      'Total_Trans_Ct', 'Total_Revolving_Bal', 'Total_Trans_Amt', 'Customer_Age', 'Months_on_book']

In [18]:
# np_la.matrix_rank(preproc_data_x.values[:100, :])
Matrix(preproc_data_x.values[:100, :]).rank()

19

# SVD - singular value decomposition

In [25]:
U, S, VT = np_la.svd(x, "econ")

# PCA (Principal component analysis)
Approach 2. Using Eigenvalues

In [52]:
print(f"features number: {x.shape[1]}; rank of data: {np_la.matrix_rank(x)}")

features number: 19; rank of data: 18


according to fact that matrix rank less by 1 to number of features,
we have only one couple of linearly dependent vectors(features)

In [72]:
lin_dep_vec_idx = None
for i in range(x.shape[1]):
    x_mod = x[:18, [j for j in range(x.shape[1]) if j != i]]
    if np.abs(np_la.det(x_mod)) > 10**-6:
        lin_dep_vec_idx = i
        break

In [90]:
lin_dep_vec_idx
x_mod = x[:, [j for j in range(x.shape[1]) if j != lin_dep_vec_idx]]
np_la.det(x_mod[:18,:])

-211827.5023703067

In [91]:
cov_x_matrix = np.cov(x_mod)

In [75]:
L, Q = np_la.eig(cov_x_matrix)

In [84]:
Q_inv = np_la.inv(Q)

In [94]:
[float(i) for i in L]

Wall time: 10 ms


<timed eval>:1: ComplexWarning: Casting complex values to real discards the imaginary part


[1502.5907058818539,
 1266.5049310682793,
 1052.2026436613387,
 778.4121775099842,
 764.7128771253768,
 631.0130656580066,
 608.222897748221,
 594.4013919884444,
 573.5804139367895,
 549.3274144608085,
 501.2348052033951,
 361.93463019861747,
 250.08265059533863,
 146.41951771063498,
 124.7092790534619,
 108.44946231229522,
 98.8426656516038,
 -9.4176528482174e-14,
 -9.4176528482174e-14,
 -1.3039506639771872e-13,
 -1.3039506639771872e-13,
 -1.7373100735175094e-13,
 -1.7373100735175094e-13,
 1.6789187974819057e-13,
 1.6789187974819057e-13,
 2.0755575828011687e-14,
 2.0755575828011687e-14,
 9.53279057850125e-14,
 9.53279057850125e-14,
 1.6064962700807223e-13,
 1.0860741825431333e-13,
 -1.4979729254895396e-14,
 -1.4979729254895396e-14,
 -9.70622472218196e-14,
 -7.463969556059899e-14,
 4.398095197484344e-14,
 4.398095197484344e-14,
 -4.038726990647483e-14,
 3.455249751567226e-14,
 2.159869904588605e-15,
 2.159869904588605e-15,
 1.6313912733674656e-14,
 1.6313912733674656e-14,
 -2.391175621

In [56]:
np_la.det(cov_x_matrix)

0.0

In [71]:
# np_la.det(x[:18,[j for j in range(x.shape[1]) if j != 13]])
np_la.matrix_rank(x[:19,:])

18